# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import datetime
import os
import random as rn

import numpy as np
import tensorflow as tf
from PIL import Image

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
rn.seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open("./Data/train.csv").readlines())
val_doc = np.random.permutation(open("./Data/val.csv").readlines())
batch_size = 4

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with some of the parts of the generator function such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size, image_size, img_idx, num_classes):
    w, h = image_size

    print("Source path = ", source_path, "; batch size =", batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t) / batch_size)
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size, len(img_idx), w, h, 3))
            batch_labels = np.zeros((batch_size, num_classes))
            for folder in range(batch_size):
                imgs = os.listdir(
                    source_path + "/" + t[folder + (batch * batch_size)].split(";")[0]
                )
                for idx, item in enumerate(img_idx):
                    image = Image.open(
                        source_path
                        + "/"
                        + t[folder + (batch * batch_size)].strip().split(";")[0]
                        + "/"
                        + imgs[item]
                    )

                    if image.size[1] == 160:
                        image = image.resize(image_size)
                    else:
                        image = image.resize(image_size)

                    image = np.array(image).astype(np.float32)

                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] - 104
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] - 117
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] - 123

                batch_labels[
                    folder, int(t[folder + (batch * batch_size)].strip().split(";")[2])
                ] = 1
            yield batch_data, batch_labels

        if (len(t) % batch_size) != 0:
            batch_data = np.zeros((len(t) % batch_size, len(img_idx), w, h, 3))
            batch_labels = np.zeros((len(t) % batch_size, num_classes))
            for folder in range(len(t) % batch_size):
                imgs = os.listdir(
                    source_path
                    + "/"
                    + t[folder + (num_batches * batch_size)].split(";")[0]
                )
                for idx, item in enumerate(img_idx):
                    image = Image.open(
                        source_path
                        + "/"
                        + t[folder + (num_batches * batch_size)].strip().split(";")[0]
                        + "/"
                        + imgs[item]
                    )
                    if image.size[1] == 160:
                        image = image.resize(image_size)
                    else:
                        image = image.resize(image_size)

                    image = np.array(image).astype(np.float32)

                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] - 104
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] - 117
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] - 123

                batch_labels[
                    folder,
                    int(t[folder + (num_batches * batch_size)].strip().split(";")[2]),
                ] = 1

            yield batch_data, batch_labels

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = "./Data/train"
val_path = "./Data/val"
num_train_sequences = len(train_doc)
print("# training sequences =", num_train_sequences)
num_val_sequences = len(val_doc)
print("# validation sequences =", num_val_sequences)
num_epochs = 70
print("# epochs =", num_epochs)
image_size = (120, 120)
print("# image_size =", image_size)
num_classes = 5
print("# num_classes =", num_classes)
img_idx = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
print("# img_idx =", img_idx)

# training sequences = 663
# validation sequences = 100
# epochs = 70
# image_size = (120, 120)
# num_classes = 5
# img_idx = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]


## Model
Here you make the model using different functionalities that Keras provides. You might want to use `TimeDistributed`, `GRU` and other RNN structures after doing transfer learning. Also remember that the last layer is the softmax. Remember that the network is designed in such a way that the model is able to fit in the memory of the webcam.

In [12]:
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import GRU, Dense, Dropout, Flatten, TimeDistributed
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model, Sequential

w, h = image_size

base_model = VGG16(include_top=False, weights="imagenet", input_shape=(w, h, 3))
x = base_model.output
x = Flatten()(x)
# x.add(Dropout(0.5))
features = Dense(64, activation="relu")(x)
conv_model = Model(inputs=base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(TimeDistributed(conv_model, input_shape=(15, 120, 120, 3)))
model.add(GRU(32, return_sequences=True))
model.add(GRU(16))
model.add(Dropout(0.5))
model.add(Dense(8, activation="relu"))
model.add(Dense(num_classes))

ModuleNotFoundError: No module named 'tensorflow.contrib'

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
adam = optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam,
    loss=CategoricalCrossentropy(from_logits=True),
    metrics=["categorical_accuracy"],
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 15, 64)           15009664  
 ibuted)                                                         
                                                                 
 gru (GRU)                   (None, 15, 32)            9408      
                                                                 
 gru_1 (GRU)                 (None, 16)                2400      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 5)                 45        
                                                        

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(
    train_path, train_doc, batch_size, image_size, img_idx, num_classes
)
val_generator = generator(
    val_path, val_doc, batch_size, image_size, img_idx, num_classes
)

In [9]:
model_name = (
    "./CheckPoints/"
    + "model_init_conv_lstm"
    + "_"
    + str(curr_dt_time).replace(" ", "").replace(":", "_")
    + "/"
)

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = (
    model_name
    + "model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5"
)

checkpoint = ModelCheckpoint(
    filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
)

LR = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
    mode="min",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.00001,
)
callbacks_list = [checkpoint, LR]

In [10]:
if (num_train_sequences % batch_size) == 0:
    steps_per_epoch = int(num_train_sequences / batch_size)
else:
    steps_per_epoch = (num_train_sequences // batch_size) + 1

if (num_val_sequences % batch_size) == 0:
    validation_steps = int(num_val_sequences / batch_size)
else:
    validation_steps = (num_val_sequences // batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [11]:
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
    callbacks=callbacks_list,
    validation_data=val_generator,
    validation_steps=validation_steps,
    class_weight=None,
    workers=1,
    initial_epoch=0,
)

Source path =  ./Data/train ; batch size = 4
Epoch 1/70


InvalidArgumentError: Graph execution error:

No OpKernel was registered to support Op 'CudnnRNN' used by {{node CudnnRNN}} with these attrs: [seed=0, dropout=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="gru", seed2=0, is_training=true]
Registered devices: [CPU, GPU]
Registered kernels:
  <no registered kernels>

	 [[CudnnRNN]]
	 [[sequential/gru/PartitionedCall]] [Op:__inference_train_function_6857]